<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_sif_freq/EqualTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Prepare

In [1]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=e870b80190caa0491d152f352a399215d01c34c30ff851bd0976dccf6dd7597e
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=4c7bbb70597a6c99158fdd18021d531a62ef611555abdf8135f518e01cd43c07
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=603b58dfa1a82cc3ff844a30714260f20693a994f41165e4d3fe356457b7e0fc
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=446f2488247050f080d94149c025f311dddfb8771d2748390ad2cd9f8ab4ae68
  Stor

In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 233 (delta 122), reused 150 (delta 55), pack-reused 0
Receiving objects: 100% (233/233), 5.87 MiB | 9.27 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/bachelor_design


In [3]:
!git checkout equal_sif_freq
!git pull

Branch 'equal_sif_freq' set up to track remote branch 'equal_sif_freq' from 'origin'.
Switched to a new branch 'equal_sif_freq'
Already up to date.


In [4]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


# Data Prepare

Origin Data

In [0]:
from cut_and_tag import load_stopwords, cut_and_remove_stopwords, cut_and_tag
from preprocess import seq2str
import config
import json

In [6]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH, 
    addh + config.STOPWORDS_PATH
)

equal_strs = None
with open(addh + config.EQUAL_DATA_PATH, "r") as fd:
    equal_strs = json.load(fd)
    
stopwords = load_stopwords(addh + config.STOPWORDS_PATH)
equal_cut_seqs = []
equal_seqs = []
for equal_str in equal_strs:
    equal_cut = cut_and_remove_stopwords(equal_str, stopwords)
    equal_cut_seqs.append(equal_cut)
    
    equal_seq = []
    for w in equal_cut:
        for c in w:
            equal_seq.append(c)
    equal_seqs.append(equal_seq)
    
origin_seqs = char_seqs[:len(equal_seqs)]
origin_cut_seqs = cut_seqs[:len(equal_seqs)]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.776 seconds.
Prefix dict has been built successfully.


# Param Load

In [0]:
import json
params = None
with open(addh + config.EQUAL_PARAM_PATH, "r") as fd:
    params = json.load(fd)

# Train

Since our embedding is designed for batch work, it should be better if we combine origin_seqs and equal_seqs.

In [0]:
origin_num = len(origin_seqs)
all_char_seqs = []
all_char_seqs.extend(origin_seqs)
all_char_seqs.extend(equal_seqs)
all_cut_seqs = []
all_cut_seqs.extend(origin_cut_seqs)
all_cut_seqs.extend(equal_cut_seqs)

In [9]:
from char_emb import CharEmbedder
from SIF import SIF
from dist_cal import DistCal
from tqdm.notebook import trange, tqdm

Using TensorFlow backend.


In [10]:
train_rec = {}
char_embedder = CharEmbedder()
sif = SIF(params["sif_alpha"])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [11]:
for dist_method in tqdm(DistCal.methods, desc="Distance"):
    dist_theta = params.get(dist_method + "_theta", None)
    for emb_method in tqdm(CharEmbedder.methods, desc="Char embed", leave=False):
        # Embedding
        all_char_emb_seqs = char_embedder.embed(all_char_seqs, emb_method)
        all_sen_vecs = sif.compose(all_cut_seqs, all_char_seqs, all_char_emb_seqs)
        
        # Split
        origin_sen_vecs = all_sen_vecs[:origin_num]
        equal_sen_vecs = all_sen_vecs[origin_num:]
        if len(origin_sen_vecs) != len(equal_sen_vecs):
            raise Exception("Length should be the same.")

        # Init distance calculater
        dist_cal = DistCal(all_sen_vecs)
            
        # Train theta
        ## Set train params
        epoch = 10000
        delta = None
        min_delta = 10**(-7)
        
        ## Use two experiments
        ## * Check whether origin and equal are similiar
        ## * Check whether origins are different
        N = len(origin_sen_vecs)
        similiar_total = N # TP + FN
        different_total = N * (N -1)/2 # FP + TN
        
        similiar_count = 0 # TP
        different_count = 0 # TN
        last_delta = 0
        epoch_range = range(epoch)
        with tqdm(epoch_range, desc="Train theta", leave=False) as epoch_tqdm:
            for epoch_count in epoch_range:
                # Do experiments
                similiar_count = 0
                for origin_sen, equal_sen in zip(origin_sen_vecs, equal_sen_vecs):
                    dist = dist_cal.cal(origin_sen, equal_sen, dist_method)
                    if dist_theta == None: # When dist_theta is not set, take the first dist as its initial value
                        dist_theta = dist
                    if dist < dist_theta:
                        similiar_count += 1

                different_count = 0
                for i in range(N-1):
                    for j in range(i+1, N):
                        dist = dist_cal.cal(origin_sen_vecs[i], origin_sen_vecs[j], dist_method)
                        if dist > dist_theta:
                            different_count += 1

                similiar_rate = similiar_count / similiar_total
                different_rate = different_count / different_total

                # Finetune theta
                if delta == None:
                    delta = dist_theta/10
                  
                now_delta = 0
                if similiar_rate > different_rate:
                    now_delta = -delta
                elif similiar_rate < different_rate:
                    now_delta = delta
                else:
                    now_delta = -last_delta

                if now_delta == -last_delta:
                    delta /= 10
                    now_delta /= 10
                dist_theta += now_delta
                last_delta = now_delta

                if dist_theta <= 0:
                    dist_theta = 0

                # Update progroess info
                epoch_tqdm.set_description("Train theta-Epoch %i" % epoch_count)
                epoch_tqdm.set_postfix(
                    similiar_rate=similiar_rate, 
                    different_rate=different_rate,
                    dist_theta=dist_theta,
                    delta=delta
                )
                epoch_tqdm.update(epoch_count+1)

                # Stop when delta's precision is enough
                if delta <= min_delta:
                    epoch_tqdm.update(epoch_range[-1] + 1)
                    break
        
        # Cache train records
        if not train_rec.get(dist_method, None):
            train_rec[dist_method] = {}
        train_rec[dist_method][emb_method] = {
            "TP": similiar_count,
            "FP": different_total - different_count,
            "FN": similiar_total - similiar_count,
            "TN": different_count,
            "dist_theta": dist_theta
        }

# Save train record

In [0]:
with open(addh + config.EQUAL_TRAIN_REC_PATH, "w") as fd:
    json.dump(train_rec, fd, indent=4)